In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
import scipy
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox, TextArea
import pandas as pd
import numpy as np

In [ ]:
x_train = pd.read_csv('fashionmnist/fashion-mnist_train.csv')
x_test = pd.read_csv('fashionmnist/fashion-mnist_test.csv')

y_train = x_train.pop('label')
y_test = x_test.pop('label')

x_train = np.array(x_train).reshape(-1, 28, 28) / 255.0
x_test = np.array(x_test).reshape(-1, 28, 28) / 255.0

In [ ]:
labeldict = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

fig, ax = plt.subplots(1, 10, figsize=(20, 2))
for i in range(10):
    ax[i].imshow(x_train[i], cmap='gray')
    ax[i].set_title(labeldict[y_train[i]])

In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# input layer
input_layer = Input(shape=(28, 28, 1))

# encoding architecture
encoded_layer1 = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoded_layer1 = MaxPool2D( (2, 2), padding='same')(encoded_layer1)
encoded_layer2 = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded_layer1)
encoded_layer2 = MaxPool2D( (2, 2), padding='same')(encoded_layer2)
encoded_layer3 = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded_layer2)
latent_view    = MaxPool2D( (2, 2), padding='same')(encoded_layer3)

# decoding architecture
decoded_layer1 = Conv2D(16, (3, 3), activation='relu', padding='same')(latent_view)
decoded_layer1 = UpSampling2D((2, 2))(decoded_layer1)
decoded_layer2 = Conv2D(32, (3, 3), activation='relu', padding='same')(decoded_layer1)
decoded_layer2 = UpSampling2D((2, 2))(decoded_layer2)
decoded_layer3 = Conv2D(64, (3, 3), activation='relu')(decoded_layer2)
decoded_layer3 = UpSampling2D((2, 2))(decoded_layer3)
output_layer   = Conv2D(1, (3, 3), padding='same')(decoded_layer3)

# compile the model
model = Model(input_layer, output_layer)
model.compile(optimizer='adam', loss='mse',metrics=['accuracy','mse'])


print(model.summary())

clr = ReduceLROnPlateau(monitor='loss',factor=0.5,patience=3,min_delta=0.01,cooldown=0,min_lr=1e-7,verbose=1)

model.fit(x_train,x_train,batch_size=2048,epochs=5,shuffle=True,validation_data=(x_test, x_test),callbacks=[clr])

In [ ]:
preds = model.predict(x_test)

In [ ]:
from PIL import Image 
f, ax = plt.subplots(1,5)
f.set_size_inches(80, 40)
for i in range(5):
    ax[i].imshow(x_train[i].reshape(28, 28))
plt.show()

In [ ]:
from imgaug import augmenters
noise = augmenters.SaltAndPepper(0.1)
seq_object = augmenters.Sequential([noise])
val_x_n = seq_object.augment_images(x_train * 255) / 255
preds = model.predict(val_x_n[:10])
f, ax = plt.subplots(1,5)
f.set_size_inches(80, 40)
for i in range(5):
    ax[i-5].imshow(preds[i].reshape(28, 28))
plt.show()

In [ ]:
fashion_train = pd.read_csv("fashionmnist/fashion-mnist_train.csv")

In [ ]:
data = fashion_train.iloc[:, 1:].values.astype(np.float32)
target = fashion_train['label'].values

In [ ]:
import umap
reduce = umap.UMAP(random_state = 223)
embedding = reduce.fit_transform(data)

In [ ]:

df = pd.DataFrame(embedding, columns=('x', 'y'))
df["class"] = target
labels = { 0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 
          5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8 : 'Bag', 9 : 'Ankle boot'}

df["class"].replace(labels, inplace=True)

In [ ]:
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})
#adjusting plot dots with plot_kws
ax = sns.pairplot(x_vars = ["x"], y_vars = ["y"],data = df, 
             hue = "class",size=11, plot_kws={"s": 4});
ax.fig.suptitle('Fashion MNIST clustered with UMAP') ;

In [ ]:
idx = np.random.choice(len(x_test), 10000)
inputs = x_test[idx]
fig, ax = plt.subplots(figsize=(10, 7))
ax.set_title('AUTOENCODERS')
coords = encoder.predict(inputs)
classes = y_test[idx]
plt.scatter(coords[:, 0], coords[:, 1], c=classes)
plt.colorbar()
for i in range(10):
    class_center = np.mean(coords[classes == i], axis=0)
    text = TextArea('{} ({})'.format(labeldict[i], i))
    ab = AnnotationBbox(text, class_center, xycoords='data', frameon=True)
    ax.add_artist(ab)
plt.show()

In [ ]:
idx = np.random.choice(len(x_test), 2000)
inputs = x_test[idx]
fig, ax = plt.subplots(figsize=(10, 7))
ax.set_title('t-SNE')
coords = TSNE(n_components=2).fit_transform(inputs.reshape(2000, -1))
classes = y_test[idx]
plt.scatter(coords[:, 0], coords[:, 1], c=classes)
plt.colorbar()
for i in range(10):
    class_center = np.mean(coords[classes == i], axis=0)
    text = TextArea('{} ({})'.format(labeldict[i], i))
    ab = AnnotationBbox(text, class_center, xycoords='data', frameon=True)
    ax.add_artist(ab)
plt.show()